In [1]:
import pickle

from gpt_ner_api_codes.prompts_all import *

import nltk
nltk.download('punkt')
%reload_ext autoreload
%autoreload 2

import tqdm
def nop(it, *a, **k):
    return it
tqdm.tqdm = nop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z004r5cc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path_original_sentence = "./data/thin-film-technology-dataset/thin_film_head_technology_total_original_sentence.jsonl"
path_entities = "./data/thin-film-technology-dataset/thin_film_head_technology_total_entities.jsonl"
path_tokens_tags = "./data/thin-film-technology-dataset/thin_film_head_technology_total.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 10, demon_size = 50, stratify = True)

(array(['AnnotatorNotes', 'Attribution', 'Component', 'Consequence',
       'Effect', 'EnergyFlow', 'Function', 'InfoFlow', 'Location',
       'Material', 'Measure', 'PhysicsFlow', 'ScientificConcept', 'Shape',
       'State', 'System', 'Value'], dtype='<U1679'), array([   25,  1908, 12911,   132,   750,  1573,  1542,   203,  2601,
        1683,   152,   147,   711,  1074,    40,  1134,   312],
      dtype=int64))
Duplicated few-shot examples:  51
length of stratified_sentences_develop:  51
length of stratified_sentences_test:  10
length of stratified_sentences_train:  3760


In [3]:
# get the prompt with few-shot learning examples, before running GPT API
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "film", num_examples=5, stratify=False)

In [4]:
print(len(all_fewshot_sen) ) # The first dimension is input sentence
print(len(all_fewshot_sen[0])) # The second dimension is few shot examples for each dimension
print(all_fewshot_sen[0][0:3])
print(len(all_fewshot_dem) ) # The first dimension is input sentence
print(len(all_fewshot_dem[0])) # The second dimension is few shot examples for each dimension
print(all_fewshot_dem[0][0:3])

10
5
['A method of eliminating a wiggle noise generated by a data write current applied to a thin-film head during a data write operation in a disk drive storage apparatus having a storage medium on which data is recorded , the method comprising the step of : on occurrence of any read error owing to the wiggle noise , causing the head to perform a dummy write operation onto a given one of one or more intersector gap intervals in a corresponding track on the storage medium ; and retrying a data read operation , subsequent to searching a target sector , the data read operation being repeated by predetermined retry times if the read error is continuously detected on the data read operation . \n', 'A thin-film single magnetic pole head which assures highly efficient magnetic energization of a main magnetic pole and which exhibits optimum recording characteristics even in case of recording on a double-layer perpendicular magnetic recording medium with a high frequency . \n', 'A thin-film ma

### The way below can only read text file 

In [5]:
with open("data/immutable_thf_not_stratified_input_10_example_5.txt", 'w') as f:
    for line in all_fewshot_dem:
        # print(len(line))
        f.write(str(line) + '\n')

with open("data/immutable_thf_not_stratified_input_10_example_5.txt", 'r') as f:
    read_all_fewshot_dem = [ line.rstrip('\n') for line in f]

print( len(read_all_fewshot_dem), len(read_all_fewshot_dem[0])  )
print(read_all_fewshot_dem[0][0])

10 1587
[


### The way below can read list successfully 

In [6]:
with open("data/immutable_thf_not_stratified_input_10_example_5.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

with open("data/immutable_thf_not_stratified_input_10_example_5.txt", "rb") as data:   # Unpickling
    read_all_fewshot_dem = pickle.load(data)

print( len(read_all_fewshot_dem), len(read_all_fewshot_dem[0])  )
print(read_all_fewshot_dem[0][0])

10 5
wiggle noise: ScientificConcept
data write current: EnergyFlow
thin-film head: Component
data write operation: Function
disk drive storage apparatus: Component
storage medium: Material
read error: Consequence
wiggle noise: ScientificConcept
head: Component
dummy write operation: Function
intersector gap intervals: Shape
corresponding track: Component
storage medium: Material
data read operation: Function
target sector: Location
data read operation: Function
predetermined retry times: Measure
read error: Consequence
data read operation: Function




# Formal data processing begins below.

In [7]:
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "film", num_examples=20, stratify=True)

Duplicated few-shot examples:  17
Length of stratified_sentences_train:  14
Considering num_examples: 14 Num_examples:  20


### For stratified dataset, we select samples based on the entity. If we take thin film head technology dataset as an example, there are 15 entities in a single round. If we pick 5 sentences, we actually select 5 entities from np.unique(development_dataset) in order.

### We must be careful when deciding the number of examples in our experiments, especially when it is not n times of number of entities.
# 4 parameters 
#### dataset
#### method
#### stratified or not
#### num_examples, size of input, (development==300) is unchanged.

# Thin film head technology dataset

In [16]:
path_original_sentence = "./data/thin-film-technology-dataset/thin_film_head_technology_total_original_sentence.jsonl"
path_entities = "./data/thin-film-technology-dataset/thin_film_head_technology_total_entities.jsonl"
path_tokens_tags = "./data/thin-film-technology-dataset/thin_film_head_technology_total.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['AnnotatorNotes', 'Attribution', 'Component', 'Consequence',
       'Effect', 'EnergyFlow', 'Function', 'InfoFlow', 'Location',
       'Material', 'Measure', 'PhysicsFlow', 'ScientificConcept', 'Shape',
       'State', 'System', 'Value'], dtype='<U1679'), array([   25,  1908, 12911,   132,   750,  1573,  1542,   203,  2601,
        1683,   152,   147,   711,  1074,    40,  1134,   312],
      dtype=int64))
Duplicated few-shot examples:  306
length of stratified_sentences_develop:  279
length of stratified_sentences_test:  500
length of stratified_sentences_train:  3760


In [17]:
with open("data/immutable_data_formal/sen_thf_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_input, data)
with open("data/immutable_data_formal/dem_thf_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_input_solutions, data)
with open("data/immutable_data_formal/sen_thf_train.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_train, data)  
with open("data/immutable_data_formal/dem_thf_train.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_train, data)      
    

with open("data/immutable_data_formal/sen_thf_input_30.txt", "rb") as data:   #Pickling
    read_sentences_input = pickle.load(data)
with open("data/immutable_data_formal/dem_thf_input_30.txt", "rb") as data:   #Pickling
    read_demonstrations_input_solutions = pickle.load(data)
with open("data/immutable_data_formal/sen_thf_train.txt", "rb") as data:   #Pickling
    sentences_train = pickle.load(data)
with open("data/immutable_data_formal/dem_thf_train.txt", "rb") as data:   #Pickling
    demonstrations_train = pickle.load(data)    
print(len(read_sentences_input), len(read_demonstrations_input_solutions), len(sentences_train), len(demonstrations_train))    

500 500 3760 3760


### num_examples = 20

In [18]:
# tfh, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "film", num_examples=100, stratify=False)
print( "tfh, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )

with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
# with open("data/immutable_data/thf_not_stratified_input_30_example_20_random.txt", "rb") as data:   # Unpickling
#     read_all_fewshot_dem = pickle.load(data)
# print( len(read_all_fewshot_dem), len(read_all_fewshot_dem[0])  )
# print(read_all_fewshot_dem[0][0])


# tfh, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "film", num_examples=100, stratify=True)
print( "tfh, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# tfh, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "film", num_examples=100, stratify=False)
print( "tfh, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# tfh, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "film", num_examples=100, stratify=True)
print( "tfh, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# tfh, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "film", num_examples=100, stratify=False)
print( "tfh, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# tfh, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "film", num_examples=100, stratify=True)
print( "tfh, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# tfh, tfidf half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "film", num_examples=100, stratify=False)
print( "tfh, tfidf half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# tfh, tfidf half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "film", num_examples=100, stratify=True)
print( "tfh, tfidf half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   

tfh, random, not stratified  500 100
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  73
Considering num_examples: 73 Num_examples:  100
tfh, random, stratified  500 73


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:25<00:00, 10.64s/it]


tfh, sentence, not stratified  500 100
['Component' 'Material' 'Location' 'Attribution' 'EnergyFlow' 'Function'
 'Shape' 'System' 'ScientificConcept' 'Effect' 'Value' 'Measure'
 'InfoFlow' 'PhysicsFlow' 'Consequence' 'State' 'AnnotatorNotes']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [02:37<00:00, 10.49s/it]


Component takes time:  157.39691400527954


...

Length of stratified_sentences_train:  38
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  32
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  31
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  29
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  29
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  31
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  31
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  34
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  31
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  34
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  31
Duplicated few-shot examples:  85
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  85
Length of st

tfh, sentence, stratified  500 30
tfh, tfidf, not stratified  500 100
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  29
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  44
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  41
Duplicated:  85
Unique:  40
Duplicated:  85
Unique:  32
Duplic

Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  41
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  29
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  31
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  31
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  40
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  29
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  31
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  36
Duplicated:  85
Uniq

Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  27
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  41
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  44
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  40
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  30
Duplicated:  85
Uniq

Duplicated:  85
Unique:  26
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  27
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  31
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  28
Duplicated:  85
Unique:  39
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  30
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  29
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  34
Duplicated:  85
Unique:  38
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  37
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  36
Duplicated:  85
Unique:  35
Duplicated:  85
Unique:  32
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  33
Duplicated:  85
Unique:  34
Duplicated:  85
Uniq

### num_examples = 10

In [11]:
# tfh, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "film", num_examples=10, stratify=False)
print( "tfh, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)


# tfh, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "film", num_examples=10, stratify=True)
print( "tfh, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# tfh, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "film", num_examples=10, stratify=False)
print( "tfh, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# tfh, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "film", num_examples=10, stratify=True)
print( "tfh, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# tfh, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "film", num_examples=10, stratify=False)
print( "tfh, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# tfh, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "film", num_examples=10, stratify=True)
print( "tfh, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# tfh, tfidf half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "film", num_examples=10, stratify=False)
print( "tfh, tfidf half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# tfh, tfidf half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "film", num_examples=10, stratify=True)
print( "tfh, tfidf half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_thf_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_thf_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)        

tfh, random, not stratified  30 10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  17
Considering num_examples: 10 Num_examples:  10
tfh, random, stratified  30 10


100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.07s/it]


tfh, sentence, not stratified  30 10
['Component' 'Location' 'Attribution' 'Material' 'Function' 'EnergyFlow'
 'ScientificConcept' 'Effect' 'Shape' 'System' 'Value' 'InfoFlow'
 'Measure' 'PhysicsFlow' 'Consequence' 'State' 'AnnotatorNotes']


100%|█████████████████████████████████████████████████████████████████████████████████████| 15/15 [02:42<00:00, 10.86s/it]


Component takes time:  162.9401936531067


...

100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.95s/it]


AnnotatorNotes takes time:  3.975095272064209


100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.07s/it]


Duplicated few-shot examples:  17
Length of stratified_sentences_train:  8
Considering num_examples: 8 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  8
Considering num_examples: 8 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  10
Considering num_examples: 10 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  12
Considering num_examples: 10 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  10
Considering num_examples: 10 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  9
Considering num_examples: 9 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  9
Considering num_examples: 9 Num_examples:  10
Duplicated few-shot examples:  17
Length of stratified_sentences_train:  12
Considering num_examples: 10 Num_examples:  10
Duplicated few-shot exam

Duplicated:  17
Unique:  12
considering num_examples:  10 num_examples:  10
Duplicated:  17
Unique:  10
considering num_examples:  10 num_examples:  10
tfh, tfidf half, stratified  30 9


# Assembly dataset

In [19]:
path_original_sentence = "./data/assembly_dataset/alternators-engines-gearboxes/train_original_sentence.jsonl"
path_entities = "./data/assembly_dataset/alternators-engines-gearboxes/train_entities.jsonl"
path_tokens_tags = "./data/assembly_dataset/alternators-engines-gearboxes/train.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['DIM', 'ID', 'OPER', 'PART', 'PROP', 'QTY', 'RPOS', 'TOOL', 'WGT'],
      dtype='<U650'), array([  55,  101,  735, 2528,   84,   92,  699,  108,    2], dtype=int64))
Duplicated few-shot examples:  300
length of stratified_sentences_develop:  219
length of stratified_sentences_test:  399
length of stratified_sentences_train:  776


In [20]:
with open("data/immutable_data_formal/sen_assembly_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_input, data)
with open("data/immutable_data_formal/dem_assembly_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_input_solutions, data)
with open("data/immutable_data_formal/sen_assembly_train.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_train, data)  
with open("data/immutable_data_formal/dem_assembly_train.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_train, data)     

### num_examples = 20

In [21]:
# assembly, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "assembly", num_examples=100, stratify=False)
print( "assembly, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# assembly, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "assembly", num_examples=100, stratify=True)
print( "assembly, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# assembly, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "assembly", num_examples=100, stratify=False)
print( "assembly, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# assembly, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "assembly", num_examples=100, stratify=True)
print( "assembly, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# assembly, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "assembly", num_examples=100, stratify=False)
print( "assembly, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# assembly, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "assembly", num_examples=100, stratify=True)
print( "assembly, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)  
    
    
# assembly, tfidf_half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "assembly", num_examples=100, stratify=False)
print( "assembly, tfidf_half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# assembly, tfidf_half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "assembly", num_examples=100, stratify=True)
print( "assembly, tfidf_half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)      

assembly, random, not stratified  399 100
Duplicated few-shot examples:  90
Length of stratified_sentences_train:  70
Considering num_examples: 70 Num_examples:  100
assembly, random, stratified  399 70


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:07<00:00,  9.63s/it]


assembly, sentence, not stratified  399 100
['PART' 'OPER' 'RPOS' 'TOOL' 'ID' 'QTY' 'PROP' 'DIM' 'WGT']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [02:54<00:00,  9.69s/it]


PART takes time:  174.35945296287537


...

Length of stratified_sentences_train:  34
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  38
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  35
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  46
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  34
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  48
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  38
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  32
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  36
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  36
Duplicated few-shot examples:  88
Length of st

Length of stratified_sentences_train:  40
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  36
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  39
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  35
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  34
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  51
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  33
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  36
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  47
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  37
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  35
Duplicated few-shot examples:  88
Length of stratified_sentences_train:  43
Duplicated few-shot examples:  88
Length of st

Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  33
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  34
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  34
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  35
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  46
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  46
Duplicated:  88
Uniq

Duplicated:  88
Unique:  40
assembly, tfidf, stratified  399 41
assembly, tfidf_half, not stratified  399 100
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  35
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  47
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  35
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  35
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  34
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  40
Duplicated:  88
Unique

Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  39
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  44
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  35
Duplicated:  88
Unique:  32
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  33
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  37
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  40
Duplicated:  88
Unique:  38
Duplicated:  88
Unique:  43
Duplicated:  88
Unique:  42
Duplicated:  88
Unique:  36
Duplicated:  88
Unique:  31
Duplicated:  88
Unique:  41
Duplicated:  88
Unique:  39
Duplicated:  88
Uniq

### num_examples = 10

In [ ]:
# assembly, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "assembly", num_examples=10, stratify=False)
print( "assembly, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# assembly, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "assembly", num_examples=10, stratify=True)
print( "assembly, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# assembly, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "assembly", num_examples=10, stratify=False)
print( "assembly, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# assembly, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "assembly", num_examples=10, stratify=True)
print( "assembly, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# assembly, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "assembly", num_examples=10, stratify=False)
print( "assembly, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# assembly, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "assembly", num_examples=10, stratify=True)
print( "assembly, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data) 
    
# assembly, tfidf_half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "assembly", num_examples=10, stratify=False)
print( "assembly, tfidf_half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# assembly, tfidf_half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "assembly", num_examples=10, stratify=True)
print( "assembly, tfidf_half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_assembly_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_assembly_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)      

# FabNER dataset

In [22]:
path_original_sentence = "./data/fabNER/fabner_simple_total_original_sentence.jsonl"
path_entities = "./data/fabNER/fabner_simple_total_entities.jsonl"
path_tokens_tags = "./data/fabNER/fabner_simple_total.jsonl"
sentences_dev, sentences_input, sentences_train, demonstrations_dev, demonstrations_input_solutions, demonstrations_train = split_dev_dataset_input(path_original_sentence, path_entities, path_tokens_tags, num_input = 500, demon_size = 300, stratify = True)

(array(['APPL', 'BIOP', 'CHAR', 'CONPRI', 'ENAT', 'FEAT', 'MACEQ', 'MANP',
       'MANS', 'MATE', 'PARA', 'PRO'], dtype='<U1044'), array([ 2210,   236,  2226, 21113,  1238,  2610,  2551, 10622,    92,
       12881,  3477,  5117], dtype=int64))
Duplicated few-shot examples:  312
length of stratified_sentences_develop:  305
length of stratified_sentences_test:  500
length of stratified_sentences_train:  13658


In [23]:
with open("data/immutable_data_formal/sen_fabner_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_input, data)
with open("data/immutable_data_formal/dem_fabner_input_30.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_input_solutions, data)
with open("data/immutable_data_formal/sen_fabner_train.txt", "wb") as data:   #Pickling
    pickle.dump(sentences_train, data)  
with open("data/immutable_data_formal/dem_fabner_train.txt", "wb") as data:   #Pickling
    pickle.dump(demonstrations_train, data)    

### num_examples = 20

In [24]:
# fabner, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "fabner", num_examples=100, stratify=False)
print( "fabner, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# fabner, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "fabner", num_examples=100, stratify=True)
print( "fabner, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_20_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# fabner, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "fabner", num_examples=100, stratify=False)
print( "fabner, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# fabner, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "fabner", num_examples=100, stratify=True)
print( "fabner, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_20_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# fabner, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "fabner", num_examples=100, stratify=False)
print( "fabner, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# fabner, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "fabner", num_examples=100, stratify=True)
print( "fabner, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_20_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)  
    
# fabner, tfidf_half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "fabner", num_examples=100, stratify=False)
print( "fabner, tfidf_half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# fabner, tfidf_half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "fabner", num_examples=100, stratify=True)
print( "fabner, tfidf_half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_20_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)      

fabner, random, not stratified  500 100
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  73
Considering num_examples: 73 Num_examples:  100
fabner, random, stratified  500 73


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:07<00:00,  8.40s/it]


fabner, sentence, not stratified  500 100
['CONPRI' 'MATE' 'MANP' 'PRO' 'PARA' 'CHAR' 'FEAT' 'MACEQ' 'APPL' 'ENAT'
 'MANS' 'BIOP']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:21<00:00,  9.05s/it]


CONPRI takes time:  81.43178606033325


...

Length of stratified_sentences_train:  39
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  42
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  40
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  37
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  39
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  42
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  38
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  36
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  39
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  43
Duplicated few-shot examples:  96
Length of st

Length of stratified_sentences_train:  40
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  52
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  34
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  42
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  39
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  46
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  47
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  44
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  39
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  41
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  42
Duplicated few-shot examples:  96
Length of stratified_sentences_train:  43
Duplicated few-shot examples:  96
Length of st

Duplicated:  96
Unique:  50
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  48
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  55
Duplicated:  96
Unique:  44
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  39
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  32
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  48
Duplicated:  96
Unique:  48
Duplicated:  96
Unique:  35
Duplicated:  96
Unique:  49
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  49
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  42
Duplicated:  96
Uniq

Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  35
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  32
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  44
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  34
Duplicated:  96
Unique:  39
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  45
Duplicated:  96
Unique:  49
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  39
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  39
Duplicated:  96
Unique:  39
Duplicated:  96
Uniq

Duplicated:  96
Unique:  50
Duplicated:  96
Unique:  31
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  45
Duplicated:  96
Unique:  55
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  48
Duplicated:  96
Unique:  32
Duplicated:  96
Unique:  47
Duplicated:  96
Unique:  39
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  34
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  44
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  35
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  46
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  36
Duplicated:  96
Unique:  40
Duplicated:  96
Unique:  38
Duplicated:  96
Unique:  42
Duplicated:  96
Unique:  41
Duplicated:  96
Unique:  43
Duplicated:  96
Unique:  45
Duplicated:  96
Unique:  37
Duplicated:  96
Unique:  59
Duplicated:  96
Unique:  39
Duplicated:  96
Uniq

### num_examples = 10

In [ ]:
# fabner, random, not stratified 
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "fabner", num_examples=10, stratify=False)
print( "fabner, random, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# fabner, random, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "random", dataset = "fabner", num_examples=10, stratify=True)
print( "fabner, random, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_10_random.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)

# fabner, sentence, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "fabner", num_examples=10, stratify=False)
print( "fabner, sentence, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)
    
# fabner, sentence, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "sentence", dataset = "fabner", num_examples=10, stratify=True)
print( "fabner, sentence, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_10_sentence.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)    
    
# fabner, tfidf, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "fabner", num_examples=10, stratify=False)
print( "fabner, tfidf, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# fabner, tfidf, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf", dataset = "fabner", num_examples=10, stratify=True)
print( "fabner, tfidf, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_10_tfidf.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data) 
    
# fabner, tfidf_half, not stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "fabner", num_examples=10, stratify=False)
print( "fabner, tfidf_half, not stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_not_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)   
    
# fabner, tfidf_half, stratified
all_fewshot_sen, all_fewshot_dem = get_textual_instruction_with_fewshot_examples(sentences_input, sentences_dev, demonstrations_dev, method = "tfidf_half", dataset = "fabner", num_examples=10, stratify=True)
print( "fabner, tfidf_half, stratified ", len(all_fewshot_dem), len(all_fewshot_dem[0])  )
with open("data/immutable_data_formal/sen_fabner_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_sen, data)
with open("data/immutable_data_formal/dem_fabner_stratified_input_30_example_10_tfidf_half.txt", "wb") as data:   #Pickling
    pickle.dump(all_fewshot_dem, data)     